# Using the Wikidata Query Service (WDQS)

The [Wikidata Query Service](https://query.wikidata.org) is a GUI you can use to access data in Wikidata by editing and submitting SPARQL queries. 

SPARQL is a semantic query language that allows you to extract any kind of data by making a query composed of logical combinations of **triples**.

The results are displayed as an HTML table, every query has a unique URL which can be bookmarked for later use. 

You can find more information about SPARQL in this [SPARQL wikibook](https://en.wikibooks.org/wiki/SPARQL). 

If you want to run a SPARQL query from a Jupyter Notebook you can use [Gastrodon](https://github.com/paulhoule/gastrodon) Python library.

## Contents
[1. Basics](#1.-Basics)

- [Triples](#--Triples)
- [Building a query](#--Building-a-query)
- [Basic query structure](#--Basic-query-structure)
- [Prefixes](#--Prefixes)
- [Labels service](#--Labels-service)

[2. Useful SPARQL keywords](#2.-Useful-SPARQL-keywords)

- [OPTIONAL and DISTINCT](#--OPTIONAL-and-DISTINCT)
- [FILTER](#--FILTER)
- [UNION](#--UNION)

[3. Modifiers](#3.-Modifiers)

- [GROUP BY](#--GROUP-BY)
- [HAVING](#--HAVING)
- [ORDER BY](#--ORDER-BY)
- [LIMIT](#--LIMIT)

[4. Aggregate functions](#4.-Aggregate-functions)

[5. Federated queries](#5.-Federated-queries)

[6. Visualizations](#6.-Visualizations)

## 1. Basics

### - Triples
<details>
<summary>.clickme</summary>

As we discussed in the first session, data in Wikidata is structured in statements, which are comprised by items (Q), properties (P), literal values and also qualifiers (a special kind of property) & references:

![Example of an statement for item Douglas Adams](https://upload.wikimedia.org/wikipedia/commons/thumb/a/ae/Datamodel_in_Wikidata.svg/800px-Datamodel_in_Wikidata.svg.png)

If you take part of the statement and read it aloud, *"Douglas Adams was educated at St. John's College"*, you will see a clear analogy with natural language.

The atomic data entity in Wikidata (as it adopts the [RDF](https://en.wikipedia.org/wiki/Resource_Description_Framework) model) is **Subject + Predicate + Object**, similar as the way we speak, or what is called a **semantic triple**. 

We could "translate" it as: `<Douglas Adams(Q42)><educated at(P69)><St John's College (Q691283)>`

All data in Wikidata is structured following this triple structure, and our queries in SPARQL will also use it.

### - Building a query
<details>
<summary>.clickme</summary>

So, knowing data is stored in triples, we can build our question or query in the same way.

In the first session we saw the main structure of queries and how to build a simple one like this:

![GIF showing process of building a query](https://upload.wikimedia.org/wikipedia/commons/8/84/Sparql_for_dummies_-_Poet_spouses.gif)

Any of the triple parts (Subject, Predicate or Object) may be variables, which makes the querying very versatile. You can always check the [Gentle introduction to the Wikidata Query Service](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/A_gentle_introduction_to_the_Wikidata_Query_Service) if you need to refresh the basics.

### - Basic query structure
<details>
<summary>.clickme</summary>

Let's say I want a list of universities in Switzerland. 

The query below can be read as *"Get me items which are instance of (P31) university (Q3918) and which country (P17) is Switzerland (Q39)"*.

```SPARQL
# Universities in Switzerland
SELECT ?item 
WHERE 
{
  ?item wdt:P31 wd:Q3918;
        wdt:P17 wd:Q39.
}
```
**[Try it](https://w.wiki/DF2)**

- `SELECT`: holds the variables (placeholders) we want to get back from the database, or results
- `WHERE`: holds the conditions we're using to filter the database, between curly brackets `{}`
- Comments in code are preceded by `#`
- Variables always follow the `?name` structure
- Each triple about a subject is terminated by a period `.`
- Multiple predicates about the same subject are separated by semicolons `;` and multiple objects for the same subject and predicate can be listed separated by commas `,`

This query brings back **62 results**. 

What if someone has defined an item not as an instance of University but as something that is a subclass of University, like "public university"?

To make sure we don't miss these results, in addition to `instance of (P31)`, we should also add `subclass of (P279)` an indefinite number of times to reach `university (Q3918)`. We can do this using **property paths** indicated by `wdt:P31/wdt:P279*` (the "*" means "indefinite number of times"). 

```SPARQL
# Universities in Switzerland
SELECT ?item  
WHERE 
{
  ?item wdt:P31/wdt:P279* wd:Q3918;
        wdt:P17 wd:Q39.
}
```
Now we have **71 results**.

**[Try it](https://w.wiki/DF4)**

### - Prefixes
<details>
<summary>.clickme</summary>


**What is `wd` and `wdt`?**

Subjects and predicates are always stored in Wikidata as URIs, or [Uniform Resource Identifiers](https://en.wikipedia.org/wiki/Uniform_Resource_Identifier). 

The object however can be stored as URI (for example in the statement `<Douglas Adams(Q42)><educated at(P69)><St John's College (Q691283)>`) or as a literal (like the `end time` "1974" of our example).

So in fact if I want to refer to `Douglas Adams` in my query, instead of `Q42` I should say: `https://wikidata.org/entity/Q42`

This turns to be a bit messy when building queries. WDQS makes it easier for us, pre-loading [prefixes](https://en.wikibooks.org/wiki/SPARQL/Prefixes) to refer to URIs in a simple way. Some of these prefixes are internal to Wikidata, like `wd`, `wdt`, `p`, `ps`, `bd`, and many others are commonly used external prefixes, like `rdf`, `skos`, `owl`, `schema`. 

The following example is redundant, as `wd` and `wdt` are already built into the WDQS, but we will have to incorporate prefixes in other situations, e.g. when we want to do federated queries.

```SPARQL
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wd: <http://www.wikidata.org/entity/>

# Universities in Switzerland
SELECT ?item
WHERE 
{
  ?item wdt:P31/wdt:P279* wd:Q3918;
        wdt:P17 wd:Q39.
}
```

**[Try it](https://w.wiki/DF5)**

### - Labels service
<details>
<summary>.clickme</summary>
The previous queries gave us the data we were looking for, but it was quite annoying to read, only bringing back Qids.

You can fetch the label, alias, or description of entities you query, with language fallback, using a built-in service before closing the `WHERE` clause.


```SPARQL
# Universities in Switzerland
SELECT ?item ?itemLabel 
WHERE 
{
  ?item wdt:P31/wdt:P279* wd:Q3918;
        wdt:P17 wd:Q39.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr,de,it". }
}
```

You don't have to remember the `SERVICE` line by heart: At the SPARQL editor window just press `ctrl` + `space` and you'll activate autocompletion. You'll be able to choose from the main SPARQL keywords but also prefixes.

**How does the `SERVICE` work?**

- If a variable in SELECT is named ?NAMELabel, you'll get the label for the entity in ?NAME
- If a variable in SELECT is named ?NAMEAltLabel, you'll get the alias for the entity in ?NAME
- If a variable in SELECT is named ?NAMEDescription, you'll get the alias for the entity in ?NAME

The Wikidata Query Service website auto-magically replaces `[AUTO_LANGUAGE]` with the language code of current user's interface. 

You can specify a list of languages to be used as fallback in case a label does not exist in your preferred language(s).

## 2. Useful SPARQL keywords

### - `OPTIONAL` and `DISTINCT`
<details>
<summary>.clickme</summary>

*a) You can use `DISTINCT` to remove duplicate entries, when they hold *the same data*. *

```SPARQL
# Universities in Switzerland
SELECT DISTINCT ?item ?itemLabel
WHERE 
{
  ?item wdt:P31/wdt:P279* wd:Q3918;
        wdt:P17 wd:Q39.
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
```

We have now **68 items**. 
**[Try it](https://w.wiki/DDs)**


*b) Let's say I also want to get images for each University, I'd do something like:*

```SPARQL
# Universities in Switzerland
SELECT DISTINCT ?item ?itemLabel ?pic
WHERE 
{
  ?item wdt:P31/wdt:P279* wd:Q3918;
        wdt:P17 wd:Q39;
        wdt:P18 ?pic.
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
```
**[Try it](https://w.wiki/DDv)**

But this only brings **22 results**. What is happening here?

The reason is that to match this query, the potential result (a university) must match all the conditions we listed: 
- be an instance/subclass of university
- country Switzerland
- have a picture

If any of those isn't matched the item is not part of the result. 

*c) To indicate that a condition is not mandatory but if the data is available we do want it, we use `OPTIONAL`.*

```SPARQL
# Universities in Switzerland
SELECT DISTINCT ?item ?itemLabel ?pic
WHERE 
{
  ?item wdt:P31/wdt:P279* wd:Q3918;
        wdt:P17 wd:Q39.
  OPTIONAL { ?item wdt:P18 ?pic.}
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
```
**[Try it](https://w.wiki/DDw)**

### - `FILTER`
<details>
<summary>.clickme</summary>

Let's forget about pictures for now. 

*a) I'm interested in Universities created in 1969, like EPFL.*

So I ask for all Universities with `inception date (P571)`. I really need this data so I don't make it optional and therefore I only get **47 results**. 

```SPARQL
# Universities in Switzerland
SELECT DISTINCT ?item ?itemLabel ?inceptionDate

WHERE 
{
  ?item wdt:P31/wdt:P279* wd:Q3918;
        wdt:P17 wd:Q39;
        wdt:P571 ?inceptionDate.
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
```
**[Try it](https://w.wiki/DFB)**


I can now extract the year from my variable `?inceptionDate` and using `FILTER`:

```SPARQL
# Universities in Switzerland
SELECT DISTINCT ?item ?itemLabel ?inceptionDate

WHERE 
{
  ?item wdt:P31/wdt:P279* wd:Q3918;
        wdt:P17 wd:Q39;
        wdt:P571 ?inceptionDate.
  FILTER(YEAR(?inceptionDate) = 1969).
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
```

**I get two results:**
![Swiss universities created in 1969](img/1969.png)

Note: this method isn't useful for big datasets - refer to this [alternative](https://en.wikibooks.org/wiki/SPARQL/FILTER#FILTER_on_values)

**[Try it](https://w.wiki/DE7)**


*b) What if now I want to know which universities don't have an inception date? I can use `FILTER NOT EXISTS`, which will bring back 21 results.*

```SPARQL
# Universities in Switzerland
SELECT DISTINCT ?item ?itemLabel ?inceptionDate

WHERE 
{
  ?item wdt:P31/wdt:P279* wd:Q3918;
        wdt:P17 wd:Q39.
 
  FILTER NOT EXISTS {?item wdt:P571 ?inceptionDate.}
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
```
**[Try it](https://w.wiki/DE4)**

### - `UNION`
<details>
<summary>.clickme</summary>
    
If I want not only universities but also museums with an inception date in Switzerland, I can use `{ ... } UNION { ... }`

```SPARQL
# Universities & Museums in Switzerland with inception date
SELECT DISTINCT ?item ?itemLabel ?inceptionDate

WHERE 
{
  {?item wdt:P31/wdt:P279* wd:Q3918;
        wdt:P17 wd:Q39;
        wdt:P571 ?inceptionDate.}
  UNION
  {?item wdt:P31/wdt:P279* wd:Q33506;
        wdt:P17 wd:Q39;
        wdt:P571 ?inceptionDate.}
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
```

**[Try it](https://w.wiki/DEA)**

**To sum up:**
![Mathematical joins in SPARQL](img/joins.png)

## 3. Modifiers

Modifiers are keywords that go after the `WHERE` clause.

### - `GROUP BY`
<details>
    <summary>.clickme</summary>
    
*a) Let's group all universities in Switzerland by their `located in the administrative territorial entity (P131)` property.*

This involves two steps:
- Counting items: `(COUNT(?variableWithItems) AS ?totalCount)`
- Grouping the count by `P131`: adding the clause `GROUP BY ?city ?cityLabel` in the end of the query

```sparql
# Count of universities in Switzerland grouped by administrative territorial entity 
SELECT ?city ?cityLabel (COUNT(?item) AS ?universities) 
       
WHERE
{
  ?item wdt:P31/wdt:P279* wd:Q3918;
        wdt:P17 wd:Q39;
        wdt:P131 ?city.
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". 
                          ?city rdfs:label ?cityLabel.
                          ?item  rdfs:label ?itemLabel.
                         }
}
GROUP BY ?city ?cityLabel
```
**[Try it!](https://w.wiki/DFG)**

![Count of universities in Switzerland grouped by administrative territorial entity ](univ.png)


*b) This is not bad, but it would be better if I had the names of the universities in each group.*

To do this I have to work with labels. What I'll do is use `GROUP_CONCAT` to get a concatenated string of all `?itemLabels`.

I have to add this in my `SELECT` clause: `(GROUP_CONCAT(DISTINCT ?variableStoringLabel; SEPARATOR=", ") AS ?variableStoringWholeString).`


```sparql
# Universities in Switzerland grouped by city administrative territorial entity
SELECT ?city ?cityLabel (COUNT(?item) AS ?universities) 
        (GROUP_CONCAT(DISTINCT ?itemLabel; SEPARATOR=", ") AS ?names)
WHERE
{
  ?item wdt:P31/wdt:P279* wd:Q3918;
        wdt:P17 wd:Q39;
        wdt:P131 ?city.
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". 
                          ?city rdfs:label ?cityLabel.
                          ?item  rdfs:label ?itemLabel.
                         }
}
GROUP BY ?city ?cityLabel
```

**[Try it!](https://w.wiki/DFE)**

![Universities in Switzerland grouped by administrative territorial entity](img/univ2.png)

### - `HAVING`
<details>
    <summary>.clickme</summary>

*a) Let's filter administrative territorial entities with more than 3 universities*

I can do this using `HAVING`, which will only work when used with `GROUP BY`, immediately below.


```sparql
# Universities in Switzerland grouped by city administrative territorial entity
SELECT ?city ?cityLabel (COUNT(?item) AS ?universities) 
        (GROUP_CONCAT(DISTINCT ?itemLabel; SEPARATOR=", ") AS ?names)
WHERE
{
  ?item wdt:P31/wdt:P279* wd:Q3918;
        wdt:P17 wd:Q39;
        wdt:P131 ?city.
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". 
                          ?city rdfs:label ?cityLabel.
                          ?item  rdfs:label ?itemLabel.
                         }
}
GROUP BY ?city ?cityLabel
HAVING (COUNT(?item)>3)
```

**[Try it!](https://w.wiki/DFK)**

![Administrative territorial entity in CH with more than 3 universities](img/univ3.png)

### - `ORDER BY`
<details>
    <summary>.clickme</summary>

By default `ORDER BY` will sort a `?variable` or `expression` in ascending order. You can also specify `ORDER BY DESC` to reverse it.

```sparql
# Universities in Switzerland grouped by city administrative territorial entity
SELECT ?city ?cityLabel (COUNT(?item) AS ?universities) 
        (GROUP_CONCAT(DISTINCT ?itemLabel; SEPARATOR=", ") AS ?names)
WHERE
{
  ?item wdt:P31/wdt:P279* wd:Q3918;
        wdt:P17 wd:Q39;
        wdt:P131 ?city.
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". 
                          ?city rdfs:label ?cityLabel.
                          ?item  rdfs:label ?itemLabel.
                         }
}
GROUP BY ?city ?cityLabel
HAVING (COUNT(?item)>3)
ORDER BY (?universities)
```

**[Try it](https://w.wiki/DFL)**

### - `LIMIT`
<details>
    <summary>.clickme</summary>

`LIMIT` + `number` gives back only `number` amount of results. It's useful to avoid a time out query if you don't know how much data you're dealing with, or to get a top10:

```sparql
# Top10 of university count by administrative territorial entities in CH
SELECT ?city ?cityLabel (COUNT(?item) AS ?universities) 
        (GROUP_CONCAT(DISTINCT ?itemLabel; SEPARATOR=", ") AS ?names)
WHERE
{
  ?item wdt:P31/wdt:P279* wd:Q3918;
        wdt:P17 wd:Q39;
        wdt:P131 ?city.
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". 
                          ?city rdfs:label ?cityLabel.
                          ?item  rdfs:label ?itemLabel.
                         }
}
GROUP BY ?city ?cityLabel
ORDER BY DESC (?universities)
LIMIT 10
```

**[Try it](https://w.wiki/DFM)**

## 4. Aggregate functions 
   
We saw some aggregate functions before, like `COUNT` and `GROUP_CONCAT`. But there are others:

- `SUM`, `AVG`: to be used with numbers;
- `MIN`, `MAX` : works for all value types;
- `SAMPLE` : any element.
   

When using aggregate functions of a label, all labels should be defined explicitly in the `SERVICE`.
    
So instead of the automatic `SERVICE` line:

```sparql
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
```

We have to define each variable and its label with the `rdfs:label` property:

```sparql
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". 
                          ?city rdfs:label ?cityLabel.
                          ?item  rdfs:label ?itemLabel.
                         }
```

## 5. Federated queries

One of the most interesting features of the WDQS is the ability to query multiple sources of data, not only Wikidata, at once. This is called a *federated query*.

You can find a list of supported endpoints and their manuals [here](https://www.mediawiki.org/wiki/Wikidata_Query_Service/User_Manual/SPARQL_Federation_endpoints).

***
[PageRank (PR)](https://en.wikipedia.org/wiki/PageRank) is an algorithm used by Google Search to rank web pages in their search engine results. According to Google, PageRank works by counting the number and quality of links to a page to determine a rough estimate of *how important* the website is. The underlying assumption is that more important websites are likely to receive more links from other websites.

I can query PRs using the DBPedia endpoint.
    
*a) Let's see which is the PR for universities located in Lausanne.*

- I'll have to add the corresponding prefix, in this case `PREFIX vrank:<http://purl.org/voc/vrank#>`
- And  `SERVICE <http://dbpedia.org/sparql> {?uni vrank:hasRank/vrank:rankValue ?pageRank}`

```sparql
# PageRank of universities in administrative territorial entity: Lausanne
PREFIX vrank:<http://purl.org/voc/vrank#>

SELECT DISTINCT ?uni ?uniLabel ?pageRank 
WHERE {
  ?uni wdt:P31/wdt:P279* wd:Q3918;
         wdt:P131 wd:Q807.
  
  SERVICE <http://dbpedia.org/sparql> {
    ?uni vrank:hasRank/vrank:rankValue ?pageRank
  }
  
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
  }
} 
ORDER BY DESC(?pr)
```

**[Try me](https://w.wiki/DFT)**

## 6. Visualizations

The default way of getting data from a query is in an HTML table. But according to which type of data you select, you can unlock diverse visualizations.

The default view, meaning the way in which results are displayed when we click on a link to results, can be changed by adding the following line at the beginning of the query:

`#defaultView:[NAME]`

Where `[NAME]` can be:

- ImageGrid
- Map
- LineChart
- AreaChart
- BarChart 
- ScatterChart
- BubbleChart
- TreeMap
- Tree
- Timeline
- Dimensions
- Graph

Each view has parameters you can modify, for more information check [Views](https://en.wikibooks.org/wiki/SPARQL/Views).

### Examples: image grid, maps, charts

Try activating the different views: 

```SPARQL
#Countries in European Union with Flags and Population
# to be displayed as
# 1) Table
# 2) Image Grid
# 3) Map
# 4) Bar Chart
# 5) Bubble Chart
SELECT ?country ?countryLabel (MAX(?population) AS ?Population) (SAMPLE(?flag_image) AS ?Flag) (MAX(?coordinate) AS ?coor) 
WHERE {
  wd:Q458 wdt:P150 ?country.   # European Union  contains administrative territorial entity
  OPTIONAL { ?country wdt:P41 ?flag_image. }
  OPTIONAL { ?country wdt:P36 ?capital. }
  OPTIONAL { ?country wdt:P1082 ?population. }
  OPTIONAL { ?capital wdt:P625  ?coordinate. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". 
                           ?country rdfs:label ?countryLabel             
                         }
}
GROUP BY ?country ?countryLabel
ORDER BY DESC(?Population)
```

[Try it](https://w.wiki/5o$)

### Example of timeline

```SPARQL
#defaultView:Timeline
SELECT ?item ?itemLabel ?launchdate (SAMPLE(?image) AS ?image)
WHERE
{
    ?item wdt:P31 wd:Q26529 .      # Space probe
    ?item wdt:P619 ?launchdate .
    OPTIONAL { ?item wdt:P18 ?image }
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
GROUP BY ?item ?itemLabel ?launchdate
```
**[Try it](https://w.wiki/DFc)**